Grain Pruning Method on Link Prediction  Task of Cora Dataset
----------------------------

###  A library for data loaders.

In [20]:
import warnings
warnings.filterwarnings('ignore')
import os
import os.path as osp
import copy
import time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import shutil
import numpy as np
import psutil
import itertools
import tracemalloc
import gc
import statistics as stat
import datatable as dt
import dill
import csv
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import torch
import torch.nn as nn
from torch.optim import Adam
import torch_geometric.transforms as T
from torch_geometric.seed import seed_everything as th_seed
from torch_geometric.utils import dropout_adj
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid, WebKB, WikipediaNetwork
from torch_geometric.nn import GAE, VGAE, GCNConv
from torch_geometric.seed import seed_everything as th_seed

from typing import Any, Dict, List, Tuple, Union, Optional

from torch.utils.data import DataLoader, Dataset,IterableDataset
from utils.utils import set_dirs, update_config_with_model_dims
from utils.loss_functions import  JointLoss
from torch.nn.utils.prune import global_unstructured, L1Unstructured
from utils.loss_functions import JointLoss
from utils.model_plot import save_auc_plot, save_loss_plot
from utils.model_utils import GAEWrapper
from utils.utils import set_dirs, set_seed




torch.autograd.set_detect_anomaly(True)


### Sparsity
- The sparsity is the parameter that is determines the rate of pruning across the layer. It is a value in range(0,0.1,1). This parameter is fixed for this notebook and change for remaining experiment. We determine it before training process. Here is all values of sparsities.

In [21]:
0.1, 0.2, 0.3, 0.4,  0.5, 0.6 , 0.7,  0.8, 0.9
;

''

### Functions for Measuring criterias

In [22]:


def get_num_parameters(model: nn.Module, count_nonzero_only=False) -> int:
    """
    calculate the total number of parameters of model
    :param count_nonzero_only: only count nonzero weights
    """
    num_counted_elements = 0
    for name, param in model.autoencoder.gae.encoder.linear1.named_parameters():
     
    

        if count_nonzero_only:
            num_counted_elements += param.count_nonzero()
        else:
            num_counted_elements += param.numel()
    return num_counted_elements


# Function to get CPU usage
def get_cpu_usage():
    return psutil.cpu_percent(interval=1)



# Function to approximate power consumption (Assume some average power usage per CPU percentage point)
def estimate_power_usage(cpu_usage):
    base_power_usage = 10  # Assumed base power usage in watts
    power_per_percent = 0.5  # Assumed additional watts per CPU usage percent
    return base_power_usage + (power_per_percent * cpu_usage)


### Functions for pruning and loading pruned model for reevaluation

In [24]:
## New
def fine_grained_prune(tensor: torch.Tensor, sparsity : float) -> torch.Tensor:
    """
    magnitude-based pruning for single tensor
    :param tensor: torch.(cuda.)Tensor, weight of conv/fc layer
    :param sparsity: float, pruning sparsity
        sparsity = #zeros / #elements = 1 - #nonzeros / #elements
    :return:
        torch.(cuda.)Tensor, mask for zeros
    """
    sparsity = min(max(0.0, sparsity), 1.0)
    if sparsity == 1.0:
        tensor.zero_()
        return torch.zeros_like(tensor)
    elif sparsity == 0.0:
        return torch.ones_like(tensor)

    num_elements = tensor.numel()

    num_zeros = round(num_elements * sparsity)
    importance = tensor.abs()
    threshold = importance.view(-1).kthvalue(num_zeros).values
    mask = torch.gt(importance, threshold)
    tensor.mul_(mask)

    return mask

class FineGrainedPruner:
    def __init__(self, model, sparsity_dict):
        self.masks = FineGrainedPruner.prune(model, sparsity_dict)

    @torch.no_grad()
    def apply(self, model):
        for name, param in model.autoencoder.gae.encoder.linear1.named_parameters():
            if name in self.masks:
                param *= self.masks[name]

    @staticmethod
    @torch.no_grad()
    def prune(model, sparsity_dict):
        masks = dict()
        for name, param in model.autoencoder.gae.encoder.linear1.named_parameters():
            if param.dim() > 1: # we only prune conv and fc weights
                if isinstance(sparsity_dict, dict):
                    masks[name] = fine_grained_prune(param, sparsity_dict[name])
                else:
                    assert(sparsity_dict < 1 and sparsity_dict >= 0)
                    if sparsity_dict > 0:
                        masks[name] = fine_grained_prune(param, sparsity_dict)
        return masks
 

def state_sparse_model(model, eval_acc=None, epoch=None):
    model.autoencoder.state_dict()
    #state_dict = model.state_dict()
    state_dict = model.autoencoder.state_dict()
    compressed_state = {}
    
    for k, v in state_dict.items():
        if torch.is_tensor(v):
            mask = v != 0
            if mask.any():  # Only compress if there are non-zeros
                compressed_state[k] = {
                    'shape': v.shape,
                    'values': v[mask]  # Store only non-zero values
                }
            else:
                compressed_state[k] = v  # Keep original if all zeros
        else:
            compressed_state[k] = v
    
    return {'net': compressed_state, 'epoch': epoch, 'acc': eval_acc}

def load_sparse_model(state_path, original_model):
    """
    Loads a model saved in the custom compressed format (non-zero values only).
    Reconstructs dense tensors before loading into the model.
    """
    # Load the compressed state_dict
    compressed_state = torch.load(state_path)
    compressed_weights = compressed_state['net']
    
    # Initialize a new state_dict for the original model
    #new_state_dict = original_model.state_dict()
    new_state_dict=original_model.autoencoder.state_dict()
    
    for k, v in compressed_weights.items():
        if isinstance(v, dict) and 'shape' in v and 'values' in v:
            # Reconstruct dense tensor from compressed format
            dense_tensor = torch.zeros(v['shape'], dtype=v['values'].dtype)
            mask = (dense_tensor != 0)  # All False initially
            # We need to know the positions of non-zero values (if available)
            # If indices were saved, use them; otherwise, assume sequential filling (simpler but may not match original positions)
            if 'indices' in v:
                # If you saved indices (advanced version)
                dense_tensor[v['indices']] = v['values']
            else:
                # If only values were saved (simpler version)
                # Flatten and fill non-zeros sequentially (may not match original positions)
                flat_tensor = dense_tensor.view(-1)
                flat_tensor[:len(v['values'])] = v['values']
                dense_tensor = flat_tensor.reshape(v['shape'])
            
            new_state_dict[k] = dense_tensor
        else:
            # If it's a normal tensor (e.g., biases, batch norm stats)
            new_state_dict[k] = v

    
    # Load the reconstructed state_dict
    original_model.autoencoder.load_state_dict(new_state_dict, strict=False)
    # original_model.load_state_dict(new_state_dict, strict=False)
    return original_model

def load_and_evaluate_pruned_model(config, model_path):
        
        # Instantiate the model
        model = NESS(config)
        # Load the pruned model
        sparse_model = load_sparse_model(model_path, model)
        print("Pruned model loaded.")
        return  sparse_model 

   


### Start loading data

#### functions for data loaders.

In [25]:
#In this cell there some class and function for loading dataset and preprocessing

class GraphLoader:
    """
    Data loader class for graph data.
    """

    def __init__(self, config: Dict[str, Any], dataset_name: str, kwargs: Dict[str, Any] = {}) -> None:
        """
        Initializes the GraphLoader.

        Parameters
        ----------
        config : Dict[str, Any]
            Dictionary containing options and arguments.
        dataset_name : str
            Name of the dataset to load.
        kwargs : Dict[str, Any], optional
            Dictionary for additional parameters if needed, by default {}.
        """
        # Get config
        self.config = config
        # Set the seed
        th_seed(config["seed"])
        # Set the paths
        paths = config["paths"]
        # data > dataset_name
        file_path = os.path.join(paths["data"], dataset_name)
        # Get the datasets
        self.train_data, self.validation_data, self.test_data = self.get_dataset(dataset_name, file_path)        
        

    def get_dataset(self, dataset_name: str, file_path: str) -> Tuple[Data, Data, Data]:
        """
        Returns the training, validation, and test datasets.

        Parameters
        ----------
        dataset_name : str
            Name of the dataset to load.
        file_path : str
            Path to the dataset.

        Returns
        -------
        Tuple[Data, Data, Data]
            Training, validation, and test datasets.
        """

        # Initialize Graph dataset class
        graph_dataset = GraphDataset(self.config, datadir=file_path, dataset_name=dataset_name)
        
        # Load Training, Validation, Test datasets
        train_data, val_data, test_data = graph_dataset._load_data()
        
        # Generate static subgraphs from training set
        train_data = self.generate_subgraphs(train_data)
  
        # Return
        return train_data, val_data, test_data
    
    
    def generate_subgraphs(self, train_data: Data) -> List[Data]:
        """
        Generates subgraphs from the training data.

        Parameters
        ----------
        train_data : Data
            Training data containing the graph.

        Returns
        -------
        List[Data]
            List of subgraphs generated from the training data.
        """
        # Initialize list to hold subgraphs
        subgraphs = [train_data]

        # Check if we are generating subgraphs from the graph. If False, we are in standard GAE mode
        if self.config["n_subgraphs"] > 1:
                
            # Generate subgraphs
            for i in range(self.config["n_subgraphs"]):
                
                # Change random seed
                th_seed(i)
                
                partition = 1.0/(self.config["n_subgraphs"]-i)
                
                # For the last subgraph, get 95% of the remaining graph. if num_val=1.0, RandomLinkSplit will raise error
                if partition == 1.0:
                    partition = 0.95
                    
                random_link_split = T.RandomLinkSplit(num_val=partition, 
                                                      num_test=0, 
                                                      is_undirected=True, 
                                                      split_labels=True, 
                                                      add_negative_train_samples=False)

                # get a subgraph from training data
                train_data, train_subgraph, _ = random_link_split(train_data)
                
                # Make sure that we are using only the nodes within the subgraph by overwriting the edge index 
                # with positive edge index + positive edge index reversed in direction (to make it undirected)
                pos_swapped = train_subgraph.pos_edge_label_index[[1,0],:] 
                train_subgraph.edge_index = torch.cat((train_subgraph.pos_edge_label_index, pos_swapped), dim=1)
                
                # Remove negative edge attributes. We want to sample negative samples during training
                # Masks are also not needed
                if hasattr(train_subgraph, "neg_edge_label_index"):
                    delattr(train_subgraph, "neg_edge_label_index")
                    delattr(train_subgraph, "neg_edge_label")
                    
                if hasattr(train_subgraph, "train_mask"):
                    delattr(train_subgraph, "train_mask")
                    delattr(train_subgraph, "val_mask")
                    delattr(train_subgraph, "test_mask")


                # store the sampled subgraph
                subgraphs = [train_subgraph] + subgraphs
                       
        # Change random seed back to original
        th_seed(self.config["seed"])
        
        # Return all subgraphs and original larger graph
        return subgraphs

    
def get_transform(options):
    """Splits data to train, validation and test, and moves them to the device"""
    transform = T.Compose([
        T.NormalizeFeatures(),
        T.ToDevice(options["device"]),
        T.RandomLinkSplit(num_val=0.05, 
                          num_test=0.15, 
                          is_undirected=True,
                          split_labels=True, 
                          add_negative_train_samples=False),
        ])
        
    return transform


class GraphDataset:
    """
    Dataset class for graph data format.
    """

    def __init__(self, config: Dict[str, Any], datadir: str, dataset_name: str) -> None:
        """
        Initializes the GraphDataset.

        Parameters
        ----------
        config : Dict[str, Any]
            Dictionary containing options and arguments.
        datadir : str
            The path to the data directory.
        dataset_name : str
            Name of the dataset to load.
        """
        self.config = config
        self.paths = config["paths"]
        self.dataset_name = dataset_name
        self.data_path = os.path.join(self.paths["data"], 'Planetoid')
        self.transform = get_transform(config)

        
    def _load_data(self) -> Tuple[Data, Data, Data]:
        """
        Loads one of many available datasets and returns features and labels.

        Returns
        -------
        Tuple[Data, Data, Data]
            Training, validation, and test datasets.
        """
        if self.dataset_name.lower() in ['cora', 'citeseer', 'pubmed']:
            # Get the dataset
            dataset = Planetoid(self.data_path, self.dataset_name, split="random", transform = self.transform)
        elif  self.dataset_name.lower() in ['chameleon']:
            # Get the dataset
            dataset = WikipediaNetwork(root=self.data_path, name=self.dataset_name, transform = self.transform)
        elif  self.dataset_name.lower() in ["cornell", "texas", "wisconsin"]:
            # Get the dataset
            dataset = WebKB(root=self.data_path, name=self.dataset_name, transform = self.transform) 
        else:
            print(f"Given dataset name is not found. Check for typos, or missing condition ")
            exit()
            
        # Data splits
        train_data, val_data, test_data = dataset[0]
        
        # Return
        return train_data, val_data, test_data


### Loads arguments and configuration for GNN-based encoder used in NESS.

In [26]:
import os
import pprint
from argparse import ArgumentParser
from os.path import abspath, dirname

import torch

from utils.utils import get_runtime_and_model_config, print_config


def get_arguments():
    # Initialize parser
    parser = ArgumentParser()
    # Dataset can be provided via command line
    parser.add_argument("-d", "--dataset", type=str, default="cora")
    # Encoder type
    parser.add_argument("-gnn", "--gnn", type=str, default="GNAE")
    # Random seed
    parser.add_argument("-seed", "--seed", type=int, default=57)
    # Whether to use contrastive loss
    parser.add_argument("-cl", "--cl", type=bool, default=False)
    # Whether to add noise to input
    parser.add_argument("-an", "--an", type=bool, default=True)
    # Whether to use GPU.
    parser.add_argument("-g", "--gpu", dest='gpu', action='store_true', 
                        help='Used to assign GPU as the device, assuming that GPU is available')
    
    parser.add_argument("-ng", "--no_gpu", dest='gpu', action='store_false', 
                        help='Used to assign CPU as the device')
    parser.set_defaults(gpu=True)
    
    # GPU device number as in "cuda:0". Defaul is 0.
    parser.add_argument("-dn", "--device_number", type=str, default='0', 
                        help='Defines which GPU to use. It is 0 by default')
    
    ### Set parameter for L2-Regularization ##****************
    parser.add_argument("-reg", "--l2_reg", type=float, default='0', 
                        help='Defines which GPU to use. It is 0 by default')
    parser.add_argument("-is_pruned", "--is_pruned", type=bool, default=False)
    
    
      ### Set callback for pruning ##****************
    parser.add_argument("-callbacks", "--callbacks", type=float, default=None, 
                        help='Defines which GPU to use. It is None by default')
    # Experiment number
    parser.add_argument("-ex", "--experiment", type=int, default=1)
    # Load model saved at specific epoch
    parser.add_argument("-m", "--model_at_epoch", type=int, default=None)
    
    # Return parser arguments along with the unknown ones
    args, unknown = parser.parse_known_args()
    return args


def get_config(args):
    # Load runtime config from config folder: ./config/
    config = get_runtime_and_model_config(args)
    # Define which device to use: GPU or CPU
    config["device"] = torch.device('cuda:'+args.device_number if torch.cuda.is_available() and args.gpu else 'cpu')
    # Model at specific epoch
    config["model_at_epoch"] = args.model_at_epoch
    # Indicate which device is being used
    config["l2_reg"]=args.l2_reg
    config["is_pruned"]=args.is_pruned
    config["callbacks"]=args.callbacks
    print(f"Device being used is {config['device']}")
    # Return
    return config

def print_config_summary(config, args=None):
    """Prints out summary of options and arguments used"""
    # Summarize config on the screen as a sanity check
    print(100 * "=")
    print(f"Here is the configuration being used:\n")
    print_config(config)
    print(100 * "=")
    if args is not None:
        print(f"Arguments being used:\n")
        print_config(args)
        print(100 * "=")


In [27]:
# Get parser / command line arguments
args = get_arguments()
# Get configuration file
config = get_config(args)


# By default, we are using the name of the dataset. This can be customized.
config["experiment"] = config["dataset"]

# File name to use when saving results as csv. This can be customized
config["file_name"] = config["experiment"] + "_sub" + str(config["n_subgraphs"]) + '_seed' + str(config["seed"])

Device being used is cpu


In [28]:
# Summarize config and arguments on the screen as a sanity check
print_config_summary(config, args)


Here is the configuration being used:

+-------------------+----------------------------------------------+
|     Parameter     |                    Value                     |
+===================+==============================================+
| Add noise         | True                                         |
+-------------------+----------------------------------------------+
| Aggregation       | mean                                         |
+-------------------+----------------------------------------------+
| Batch size        | 128                                          |
+-------------------+----------------------------------------------+
| Callbacks         | None                                         |
+-------------------+----------------------------------------------+
| Contrastive loss  | False                                        |
+-------------------+----------------------------------------------+
| Cosine similarity | False                                     

### NESS Class

In [29]:


class NESS:
    """
    Model: Trains a Graph Autoencoder with a Projection network, using NESS framework.
    """

    def __init__(self, config: Dict):
        """Initializes the NESS class.

        Parameters
        ----------
        config : dict
            Configuration dictionary with parameters for training a Graph Autoencoder 
            using NESS framework.
        """
        # Get config
        self.config = config
        # Set L2-Regularization Parameters******************
        self.l2_reg=config["l2_reg"]
        # Set callback for pruning
        self.callbacks=config["callbacks"]
        
        # Define which device to use: GPU, or CPU
        self.device = config["device"]
        # Create empty lists and dictionary
        self.model_dict, self.summary = {}, {}
        # Set random seed
        set_seed(self.config)
        # Set paths for results and initialize some arrays to collect data during training
        self._set_paths()
        # Set directories i.e. create ones that are missing.
        set_dirs(self.config)
        # ------Network---------
        # Instantiate networks
        print("Building the models for training and evaluation in NESS framework...")
        # Set Autoencoders i.e. setting loss, optimizer, and device assignment (GPU, or CPU)
        self.set_autoencoder()
        # Set scheduler (its use is optional)
        self._set_scheduler()
        # Print out model architecture
        self.print_model_summary()
        

        
    def set_autoencoder(self) -> None:
        """Sets up the autoencoder model, optimizer, and loss.
        
        This function is responsible for initializing the Graph Autoencoder, setting up 
        the optimizer, and defining the joint loss.
        """   
        # Instantiate the model for the text Autoencoder
        self.autoencoder = GAEWrapper(self.config)
        # Add the model and its name to a list to save, and load in the future
        self.model_dict.update({"autoencoder": self.autoencoder})
        # Assign autoencoder to a device
        for _, model in self.model_dict.items(): model.to(self.device)
        
        # Get model parameters
        parameters = [model.parameters() for _, model in self.model_dict.items()]
        
        # Joint loss including contrastive, reconstruction and distance losses
        self.joint_loss = None if self.config["dataset"][:4] == "ogbl" else JointLoss(self.config)
        
        # Set optimizer for autoencoder
        self.optimizer_ae = self._adam(parameters, lr=self.config["learning_rate"])
        
        # Add items to summary to be used for reporting later
        self.summary.update({"recon_loss": []})
        

    def set_parallelism(self, model) -> None:
        """Sets up parallelism in training if multiple GPUs are available.

        Parameters
        ----------
        model : torch.nn.Module
            The model for which the parallelism is to be set.

        Returns
        -------
        model : torch.nn.Module
            The input model wrapped in DataParallel if multiple GPUs are available.
        """
        # If we are using GPU, and if there are multiple GPUs, parallelize training
        if torch.cuda.is_available() and torch.cuda.device_count() > 1:
            print(torch.cuda.device_count(), " GPUs will be used!")
            model = torch.nn.DataParallel(model)
        return model

    def fit(self, is_pruned:False, data_loader: DataLoader, callbacks=None) -> None:
        """Fits model to the data.

        Parameters
        ----------
        data_loader : DataLoader
            The DataLoader object that provides the data.
        """
        print(f'is_pruned:{is_pruned}')
        # Get data loaders
        train_data = data_loader.train_data
        validation_data = data_loader.validation_data
        test_data = data_loader.test_data

        # Placeholders to record losses per batch
        self.metrics = {"tloss_e": [], "vloss_e": [], "rloss_e": [], "zloss_e": [], "val_auc": [], "tr_auc": []}
        self.val_auc = "NA"
        self.tr_auc = "NA"

        # Turn on training mode for the model.
        self.set_mode(mode="training")

        # Reset best test auc
        self.best_val_auc = 0
        self.best_epoch = 0
        self.patient = 0
        self.is_pruned=is_pruned

        
        # Start joint training of Autoencoder with Projection network
        for epoch in range(self.config["epochs"]):
            
            # Keep a record of epoch
            self.epoch = epoch
            
            # 0 - Update Autoencoder
            self.update_autoencoder(train_data, callbacks)
            if callbacks is not None:
                   for callback in callbacks:
                          callback()
            
            # 1 - Update log message using epoch and batch numbers
           
            if epoch % 40 == 0:
                   self.update_log(epoch)
            
            # 2 - Clean-up for efficient memory usage.
            gc.collect()                
                
            # 3 - Run Validation
            self.run_validation(is_pruned, train_data, validation_data)

            # 4 - Change learning rate if scheduler==True
            _ = self.scheduler.step() if self.config["scheduler"] else None
            
            # 5 - Stop training if we run out of patience
            if self.patient == self.config["patience"]:
                break
            
        # Get the test performance and computing inference time 
       
        start = time.time()
        self.test_auc, self.test_ap = self.autoencoder.single_test(train_data, test_data)
        end = time.time()
        self.t_inference= end-start

        # Save plots of training loss and validation auc
        save_loss_plot(self.metrics, self._plots_path)
        save_auc_plot(self.metrics, self._plots_path)
        
        # Convert loss dictionary to a dataframe
        loss_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in self.metrics.items()]))
        
        # Save loss dataframe as csv file for later use
        loss_df.to_csv(self._loss_path + "/losses.csv")
        
            
    def run_validation(self, is_pruned, train_data: Union[List, torch.Tensor], validation_data: Union[List, torch.Tensor]) -> None:
        """Runs validation on the trained model and save weights if validation AUC improves.

        Parameters
        ----------
        train_data : List or torch.Tensor
            The training dataset.

        validation_data : List or torch.Tensor
            The validation dataset.
        """
        
        # Set the evaluation mode
        self.set_mode(mode="evaluation")
        
        # Validate every nth epoch. n=1 by default, but it can be changed in the config file
        if self.config["validate"]:

            # Compute validation AUCs
            self.val_auc, _ = self.autoencoder.single_test(train_data, validation_data)
                
            # Append auc's to the list to use for plots
            self.metrics["val_auc"].append(self.val_auc)
                
        # Save intermediate model on regular intervals
        if self.epoch >=self.config["nth_epoch"] and self.epoch % self.config["nth_epoch"] == 0:
            
            # Check the test auc at this epoch
            self.config["model_at_epoch"] = self.epoch
            val_auc, _ = self.autoencoder.single_test(train_data, validation_data)

            # Update the metrics.
            if val_auc > self.best_val_auc:
                self.best_val_auc = val_auc
                self.best_epoch = self.epoch 
                self.save_weights(is_pruned)
                self.patient = 0
            else:
                self.patient += 1
                    
        # Set training mode back
        self.set_mode(mode="training")
      
    

    def update_autoencoder(self, subgraphs: List[Data], callbacks) -> None:
        """Updates autoencoder model.

        Parameters
        ----------
        subgraphs : list of Data
            A list that contains subgraphs + original training graph.
        """
        total_loss, contrastive_loss, recon_loss, zrecon_loss = [], [], [], []
        
        # Last element of the list is the original whole training graph
        graph = subgraphs[-1]
        
        # If len(subgraphs) > 1, it means that we sampled subgraphs from the graph. Else, we have a standard GAE
        subgraphs = subgraphs[:-1] if len(subgraphs) > 1 else subgraphs
        
        # A list to hold list of latents --- will be used to compute contrastive loss
        z_list = []
        
        # Initialize total loss
        tloss = None
        
        # pass subgraphs through model to reconstruct the original graph from subgraphs
        for sg in subgraphs:
            
            # Reference graph
            ref_graph = graph if self.config["full_graph"] else sg
                        
            # Drop edges if True
            if self.config["add_noise"]:
                sg.edge_index, sg.edge_attr = dropout_adj(sg.edge_index, edge_attr= sg.edge_attr, p=self.config["p_noise"])
            
            # Forwards pass
            z, latent = self.autoencoder(sg.x, sg.edge_index)           

            # Reconstruction loss by using GAE's native function
            rloss = self.autoencoder.gae.recon_loss(latent, ref_graph.pos_edge_label_index)
            
            # If the model is a variational model
            if self.autoencoder.variational:
                rloss = rloss + (1 / ref_graph.num_nodes) * self.autoencoder.gae.kl_loss()
            
            # Store z to the list
            z_list.append(z)
            
            # total loss
            tloss = tloss + rloss if tloss  is not None else rloss
            
            # Accumulate losses
            total_loss.append(tloss)
            recon_loss.append(rloss)
            
        # Clean up
        del rloss, tloss
        gc.collect()

        # Compute the losses
        n = len(total_loss)
        total_loss = sum(total_loss) / n
        recon_loss = sum(recon_loss) / n
        
        # If the graph is large such as pubmed, push the losses to cpu.
        if self.config["dataset"] == "pubmed":
            total_loss = total_loss.cpu()
            recon_loss = recon_loss.cpu()
            
        # Initiliaze contrastive loss
        closs = None
        zloss = None
        
        if self.config["contrastive_loss"] and len(subgraphs)>1:
                    
            # Generate combinations of z's to compute contrastive loss
            z_combinations = self.get_combinations_of_subgraphs(z_list)
            
            # Compute the contrastive loss for each pair of latent vectors
            for z in z_combinations:
                # Contrastive loss
                zloss = self.joint_loss(z)
                
                # Total contrastive loss
                closs = closs + zloss if closs is not None else zloss

            # Mean constrative loss
            closs = closs/len(z_combinations)
        
        # Update total loss
        total_loss = total_loss + closs if closs is not None else total_loss
        
        # Record losses
        self.metrics["tloss_e"].append(total_loss.item())
        self.metrics["rloss_e"].append(recon_loss.item())
        self.metrics["zloss_e"].append(closs.item() if closs is not None else 0)
        
        # Update Autoencoder params
        self._update_model(total_loss, self.optimizer_ae, retain_graph=True,  callbacks=None)
        
        # Delete loss and associated graph for efficient memory usage
        del total_loss, recon_loss, closs, zloss
        gc.collect()


    def get_combinations_of_subgraphs(self, z_list: List[Data]) -> List[Tuple[Data, Data]]:
        """Generates a list of combinations of subgraphs from the list of subgraphs.

        Parameters
        ----------
        z_list : list of Data
            List of subgraphs e.g. [z1, z2, z3, ...]

        Returns
        -------
        list of tuple
            A list of combinations of subgraphs e.g. [(z1, z2), (z1, z3), ...]
        """                            
        # Compute combinations of subgraphs [(z1, z2), (z1, z3)...]
        subgraph_combinations = list(itertools.combinations(z_list, 2))
        # List to store the concatenated subgraphs
        concatenated_subgraphs_list = []
        
        # Go through combinations
        for (zi, zj) in subgraph_combinations:
            # Concatenate xi, and xj, and turn it into a tensor
            z = torch.cat((zi, zj), dim=0)
            
            # Add it to the list
            concatenated_subgraphs_list.append(z)
        
        # Return the list of combination of subgraphs
        return concatenated_subgraphs_list
    
    def clean_up_memory(self, losses: List) -> None:
        """Deletes losses with attached graph, and cleans up memory.

        Parameters
        ----------
        losses : list
            List of loss values to be deleted.
        """
        for loss in losses: del loss
        gc.collect()

    def update_log(self, epoch: int) -> None:
        """Updates the messages displayed during training and evaluation.

        Parameters
        ----------
        epoch : int
            The current epoch number.
        """
        # For the first epoch, add losses for batches since we still don't have loss for the epoch
        if epoch < 1:
            description = f"Epoch:[{epoch - 1}], Total loss:{self.metrics['tloss_e'][-1]:.4f}"
            description += f", X recon loss:{self.metrics['rloss_e'][-1]:.4f}"
            if self.config["contrastive_loss"]:
                description += f", contrastive loss:{self.metrics['zloss_e'][-1]:.6f}"
            description += f", val auc:{self.val_auc}"

        # For sub-sequent epochs, display only epoch losses.
        else:
            description = f"Epoch:[{epoch - 1}] training loss:{self.metrics['tloss_e'][-1]:.4f}"
            description += f", X recon loss:{self.metrics['rloss_e'][-1]:.4f}"
            if self.config["contrastive_loss"]:
                description += f", contrastive loss:{self.metrics['zloss_e'][-1]:.6f}"
            # Add validation auc
            description += f", val auc:{self.val_auc}"

        # Update the displayed message
        print(description)

    def set_mode(self, mode: str = "training") -> None:
        """Sets the mode of the models, either as .train(), or .eval().

        Parameters
        ----------
        mode : str, optional
            Mode in which to set the models, by default "training".
        """
        for _, model in self.model_dict.items():
            model.train() if mode == "training" else model.eval()
        
            
    def save_weights(self,is_pruned:bool=False, with_epoch: bool = False) -> None:
        """Saves weights of the models.

        Parameters
        ----------
        with_epoch : bool, optional
            If True, includes the epoch number in the filename, by default False.
        """
        for model_name in self.model_dict:
            
            # Check if we want to save the model at a specific epoch
            file_name = model_name + "_" + str(self.epoch) if with_epoch else model_name
            
            # Save the model
            if is_pruned==False:
            
                dill.dump(self.model_dict[model_name], open(self._model_path + "/" + file_name + ".pt", 'wb'))
            if is_pruned==True:  
                dill.dump(self.model_dict[model_name], open(self._model_path + "/" + file_name + "pruned_fine_tuned.pt", 'wb'))
        
        print("Done with saving models.")

   

    def load_models(self, epoch: Optional[int] = None) -> None:
        """Loads weights saved at the end of the training.

        Parameters
        ----------
        epoch : int, optional
            If provided, loads the weights saved at the specified epoch, by default None.
        """
        for model_name in self.model_dict:
            print(f"is_pruned:{is_pruned}" )
            
            # Check if we want to load the model saved at a specific epoch
            file_name = model_name + "_" + str(epoch) if epoch is not None else model_name

            # Load the model
            if is_pruned==False:
                model = dill.load(open(self._model_path + "/" + file_name + ".pt", 'rb'))
            if is_pruned==True:
                
                model = dill.load(open(self._model_path + "/" + file_name + "pruned_fine_tuned.pt", 'rb'))
            # Register model to the class
            setattr(self, model_name, model.eval())
            print(f"--{model_name} is loaded")
        
        print("Done with loading models.")

    def print_model_summary(self) -> None:
        """Displays model architectures as a sanity check to see if the models are constructed correctly."""
        # Summary of the model
        description = f"{40 * '-'}Summary of the models:{40 * '-'}\n"
        description += f"{34 * '='} NESS Architecture {34 * '='}\n"
        description += f"{self.autoencoder}\n"
        # Print model architecture
        print(description)

    def _update_model(self, loss: torch.Tensor, optimizer: torch.optim.Optimizer, retain_graph: bool = True, callbacks=None) -> None:
        """Does backpropagation and updates the model parameters.

        Parameters
        ----------
        loss : torch.Tensor
            Loss containing computational graph.

        optimizer : torch.optim.Optimizer
            Optimizer used during training.

        retain_graph : bool, optional
            If True, retains the computational graph after backpropagation, by default True.
        """
        # Reset optimizer
        optimizer.zero_grad()
        # Backward propagation to compute gradients
        loss.backward(retain_graph=retain_graph)
        # Update weights
        optimizer.step()
        if self.callbacks is not None:
            for callback in self.callbacks:
                    callback()

    def _set_scheduler(self) -> None:
        """Sets a scheduler for the learning rate of the autoencoder."""
        # Set scheduler (Its use will be optional)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer_ae, step_size=1, gamma=0.99)

    def _set_paths(self) -> None:
        """Sets paths to be used for saving results at the end of the training."""
        # Top results directory
        self._results_path = os.path.join(self.config["paths"]["results"], self.config["experiment"])
        # Directory to save model
        self._model_path = os.path.join(self._results_path, "training", "model")
        # Directory to save plots as png files
        self._plots_path = os.path.join(self._results_path, "training", "plots")
        # Directory to save losses as csv file
        self._loss_path = os.path.join(self._results_path, "training", "loss")

    def _adam(self, params: Union[List, Tuple], lr: float = 1e-4,weight_decay: float = 0) -> torch.optim.AdamW:
        """Sets up AdamW optimizer using model parameters.

        Parameters
        ----------
        params : list or tuple
            Parameters of the models to optimize.

        lr : float, optional
            Learning rate, by default 1e-4.

        weight_decay: float, optional
            L2 regularization coefficient, by default 1e-5.

        Returns
        -------
        torch.optim.AdamW
            AdamW optimizer.
         """
        return torch.optim.AdamW(itertools.chain(*params), lr=lr, betas=(0.9, 0.999), eps=1e-07, weight_decay= self.l2_reg)
       

###  Functions for training dataset

In [30]:


def train( config: Dict, data_loader: IterableDataset,is_pruned:bool=False, save_weights: bool = True) -> None:
    """
    Trains the model using provided configuration and data loader.

    Parameters
    ----------
    config : dict
        Dictionary containing options.

    data_loader : IterableDataset
        Pytorch data loader used for training the model.

    save_weights : bool, optional
        If True, the trained model is saved. By default, it's True.
    
    """
    
    # Instantiate model
    model = NESS(config)
    # Start the clock to measure the training time
    start = time.process_time()
    # Fit the model to the data
    model.fit(is_pruned, data_loader)
    # Total time spent on training
    training_time = time.process_time() - start
    # Report the training time
    print("Done with training...")
    print(f"Training time:  {training_time//60} minutes, {training_time%60} seconds")
    

    
    # Return the best Test set AUC
    return model.test_auc, model.test_ap, model.t_inference, model.val_auc, model


def main(config: Dict,is_pruned:bool=False ) -> None:
    """
    The main function that starts the execution of the program. Takes the 
    configuration dictionary as input.

    Parameters
    ----------
    config : dict
        Dictionary containing options.
    """
   
    # Ser directories (or create if they don't exist)
    set_dirs(config)
    # Get data loader for first dataset.
    ds_loader = GraphLoader(config, dataset_name=config["dataset"])
    # Add the number of features in a dataset as the first dimension of the model
    config = update_config_with_model_dims(ds_loader, config)
    # Start training and save model weights at the end
    test_auc, test_ap,t_inference,val_auc, model= train( config, ds_loader, is_pruned=is_pruned,save_weights=True)
    # Return best test auc
    return test_auc, test_ap, t_inference, val_auc, model

###  Pruning the Model and Re-Evaluate the Accuracy.

In [31]:
### Settig Sparsity
sparsity=0.5

# The number of iterations
num_iterations= 2

In [32]:
import statistics as stat


# This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
Eva_final=dict()

# The following are all list of criteria for measurements. 
# We collect all desired datas of each list across iterations. 
# Then, we compute average and std of each list.



#Base model
Base_model_accuracy=[]
T_base_model=[]
Num_parm_base_model=[]
Base_model_size=[]
Base_Energy_Consumption=[]
Base_Cpu_Usage=[]
Base_Memory_Usage=[]

#Pruned model
Pruned_model_accuracy=[]
T_pruned_model=[]
Num_parm_pruned_model=[]
Pruned_model_size=[]
Pruned_Energy_Consumption=[]
Pruned_Cpu_Usage=[]
Pruned_Memory_Usage=[]

#Pruned and finetune model
Pruned_finetune_model_accuracy=[]
T_pruned_finetune_model=[]
Num_parm_pruned_finetune_model=[]
Pruned_finetune_model_size=[]
Pruned_finetune_Energy_Consumption=[]
Pruned_finetune_Cpu_Usage=[]
Pruned_finetune_Memory_Usage=[]

# Here is the dictionary to record the list of all measurements
Eva_measure={'base model accuracy':Base_model_accuracy,
            'time inference of base model':T_base_model,
            'number parmameters of base model':Num_parm_base_model,
            'base model size':Base_model_size,
            'energy consumption of base model':Base_Energy_Consumption,
            'cpu usage of base model':Base_Cpu_Usage,
            'memory usage of base model':Base_Memory_Usage,
            'pruned model accuracy': Pruned_model_accuracy,
            'time inference of pruned model':T_pruned_model,
            'number parmameters of pruned model':Num_parm_pruned_model,
            'pruned model size':Pruned_model_size,
            'energy consumption of pruned model':Pruned_Energy_Consumption,
            'cpu usage of pruned model':Pruned_Cpu_Usage,
            'memory usage of pruned model':Pruned_Memory_Usage,
            'pruned finetune model accuracy':Pruned_finetune_model_accuracy,
            'time inference of pruned finetune model':T_pruned_finetune_model,
            'number parmameters of pruned finetune model':Num_parm_pruned_finetune_model,
            'pruned finetune model size':Pruned_finetune_model_size,
            'energy consumption of pruned_finetune model':Pruned_finetune_Energy_Consumption,
            'cpu usage of pruned_finetune model':Pruned_finetune_Cpu_Usage,
            'memory usage of pruned_finetune model':Pruned_finetune_Memory_Usage}
  

In [33]:
# The Path os saved model
results_path = os.path.join(config["paths"]["results"], config["experiment"])

# Dividing dataset 
ds_loader = GraphLoader(config, dataset_name=config["dataset"])
train_data = ds_loader.train_data
validation_data = ds_loader.validation_data
test_data = ds_loader.test_data

### Training, Pruning, Finetuning

In [34]:
import statistics as stat
rate_spar=[0.1, 0.2, 0.3, 0.4,  0.5, 0.6 , 0.7,  0.8, 0.9]
for sparsity in rate_spar:

    # This is a dictionary to save all measurements. Aftre measuring, we can compute mean and std of each item.
    Eva_final=dict()

    # The following are all list of criteria for measurements. 
    # We collect all desired datas of each list across iterations. 
    # Then, we compute average and std of each list.



    #Base model
    Base_model_accuracy=[]
    T_base_model=[]
    Num_parm_base_model=[]
    Base_model_size=[]
    Base_Energy_Consumption=[]
    Base_Cpu_Usage=[]
    Base_Memory_Usage=[]

    #Pruned model
    Pruned_model_accuracy=[]
    T_pruned_model=[]
    Num_parm_pruned_model=[]
    Pruned_model_size=[]
    Pruned_Energy_Consumption=[]
    Pruned_Cpu_Usage=[]
    Pruned_Memory_Usage=[]

    #Pruned and finetune model
    Pruned_finetune_model_accuracy=[]
    T_pruned_finetune_model=[]
    Num_parm_pruned_finetune_model=[]
    Pruned_finetune_model_size=[]
    Pruned_finetune_Energy_Consumption=[]
    Pruned_finetune_Cpu_Usage=[]
    Pruned_finetune_Memory_Usage=[]

    # Here is the dictionary to record the list of all measurements
    Eva_measure={'base model accuracy':Base_model_accuracy,
                'time inference of base model':T_base_model,
                'number parmameters of base model':Num_parm_base_model,
                'base model size':Base_model_size,
                'energy consumption of base model':Base_Energy_Consumption,
                'cpu usage of base model':Base_Cpu_Usage,
                'memory usage of base model':Base_Memory_Usage,
                'pruned model accuracy': Pruned_model_accuracy,
                'time inference of pruned model':T_pruned_model,
                'number parmameters of pruned model':Num_parm_pruned_model,
                'pruned model size':Pruned_model_size,
                'energy consumption of pruned model':Pruned_Energy_Consumption,
                'cpu usage of pruned model':Pruned_Cpu_Usage,
                'memory usage of pruned model':Pruned_Memory_Usage,
                'pruned finetune model accuracy':Pruned_finetune_model_accuracy,
                'time inference of pruned finetune model':T_pruned_finetune_model,
                'number parmameters of pruned finetune model':Num_parm_pruned_finetune_model,
                'pruned finetune model size':Pruned_finetune_model_size,
                'energy consumption of pruned_finetune model':Pruned_finetune_Energy_Consumption,
                'cpu usage of pruned_finetune model':Pruned_finetune_Cpu_Usage,
                'memory usage of pruned_finetune model':Pruned_finetune_Memory_Usage}

    
    for i in range(num_iterations)  :
            
            print('________________________________________')
            print(f'This is iteration {i}')   

            # It is a dictionary to arrange output of this iteration
            Eva=dict() 

            print(f'Training and evaluation before pruning ')
            is_pruned=False
            base_model_accuracy, test_ap, t_base_model,val_auc, model = main(config,is_pruned=False)

            best_checkpoint = dict()
            best_checkpoint['state_dict'] = copy.deepcopy(model.autoencoder.state_dict())
            model.autoencoder.load_state_dict(best_checkpoint['state_dict'])
            recover_model = lambda: model.autoencoder.load_state_dict(best_checkpoint['state_dict'])


            # Start monitoring CPU and memory usage, model size, number of parametes, time inference and  power consumption
            gc.collect()
            time.sleep(5)  # Add a 5-second delay to stabilize the initial state
            tracemalloc.start()  # Start tracking memory allocations
            snapshot_before = tracemalloc.take_snapshot()#take a snapshot of the current memory state before starting the measurement.

            t0 = time.perf_counter()
            initial_cpu_usage = get_cpu_usage()
            power_usage = estimate_power_usage(initial_cpu_usage)

            test_auc, test_ap = model.autoencoder.single_test(train_data, test_data)
            base_model_accuracy =test_auc


            base_cpu_usage = get_cpu_usage()
            t1 = time.perf_counter()
            t_base_model=t1-t0

            snapshot_after = tracemalloc.take_snapshot()
            tracemalloc.stop()
            top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

            base_total_memory_diff = sum([stat.size_diff for stat in top_stats])
            base_energy_consumption = power_usage * t_base_model
            num_parm_base_model=get_num_parameters(model, count_nonzero_only=True)

            # Base Model Size
            model_path = os.path.join(results_path, "training", "model", "autoencoder.pt")
            base_model_size = os.path.getsize(model_path)

            gc.collect()
            time.sleep(5) 

            print(f'*****Results of base model*********')

            print(f"base model has accuracy on test set={base_model_accuracy:.2f}%")
            print(f"base model has size={base_model_size:.2f} bit")
            print(f"The time inference of base model is ={t_base_model}") 
            print(f"The number of parametrs of base model is:{num_parm_base_model}") 

            print(f"Energy Consumption : {base_energy_consumption:.3f}")
            print(f"total memory usage of base model':{base_total_memory_diff} ")
            print(f"cpu usage of base model':{base_cpu_usage:.3f} %")


            #Update Eva dictionary
            Eva.update({'base model accuracy': base_model_accuracy,
                        'time inference of base model': t_base_model,
                        'number parmameters of base model': num_parm_base_model,
                        'size of base model': base_model_size, 
                        'energy consumption of base model':base_energy_consumption,
                        'total memory usage of base model':base_total_memory_diff,
                        'cpu usage of base model':base_cpu_usage
                       })



            print('_______________________________________________________')
            print(f'Prune the Model and Re-Evaluate the Accuracy')

            print('_________******************************_____________')
            print(f'Pruning the Model')

            recover_model()
            model.save_weights()
            model.load_models()

            # The path of pruned model
            pruned_model_path = os.path.join(results_path, "training", "model", "autoencoder" + "pruned.pt")
            # Applying Pruning method
            pruner = FineGrainedPruner(model, sparsity)
            # Remove zero weights 
            state = state_sparse_model(model)
            # Saving pruned model on disk
            torch.save(state, pruned_model_path)

            # Reload Pruned model from disk
            pruned_model=load_and_evaluate_pruned_model(config,pruned_model_path)


            print('****************Result of pruning ******************')

            gc.collect()
            time.sleep(5)  
            tracemalloc.start()  
            snapshot_before = tracemalloc.take_snapshot()

            t0 = time.perf_counter()
            initial_cpu_usage = get_cpu_usage()
            power_usage = estimate_power_usage(initial_cpu_usage)
            # accuracy
            pruned_model_accuracy, test_ap=pruned_model.autoencoder.single_test(train_data, test_data)

            pruned_cpu_usage = get_cpu_usage()
            t1 = time.perf_counter()
            t_pruned_model=t1-t0

            snapshot_after = tracemalloc.take_snapshot()
            tracemalloc.stop()
            top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

            pruned_total_memory_diff = sum([stat.size_diff for stat in top_stats])
            pruned_energy_consumption = power_usage * t_pruned_model

            pruned_model_size = os.path.getsize(pruned_model_path )
            num_parm_pruned_model=get_num_parameters(pruned_model, count_nonzero_only=True)

            gc.collect()
            time.sleep(5)  

            ###### Report of pruning 
            print(f"pruned model has accuracy on test set={pruned_model_accuracy:.2f}%")
            print(f"pruned model has size={pruned_model_size:.2f}")
            print(f"The time inference of pruned model is ={t_pruned_model}") 
            print(f"The number of parametrs of pruned model is:{num_parm_pruned_model}") 

            print(f"Energy Consumption : {pruned_energy_consumption:.3f}")
            print(f"total memory usage of pruned model':{pruned_total_memory_diff} ")
            print(f"cpu usage of pruned model':{pruned_cpu_usage:.3f} %")


            #Update Eva dictionary
            Eva.update({'pruned model accuracy': pruned_model_accuracy,
                        'time inference of pruned model': t_pruned_model,
                        'number parmameters of pruned model': num_parm_pruned_model,
                        'size of pruned model': pruned_model_size, 
                        'energy consumption of pruned model':pruned_energy_consumption,
                        'total memory usage of pruned model':pruned_total_memory_diff,
                        'cpu usage of pruned model':pruned_cpu_usage
                       })

            gc.collect()
            time.sleep(5) 

            print('________*******************************_____________')
            print(f'Finetuning Pruned Sparse Model')

            is_prund=True
            model.save_weights()
            model.load_models()
             # Callbacks: It prunes model after trainig in each epoch
            callbacks=[lambda:FineGrainedPruner(model, sparsity)]
             # Finetuning
            model.fit(is_prund,ds_loader,callbacks)
            fine_tuned_state=state_sparse_model(model)
            # Fine-tuned model path
            pruned_finetune_model_path = os.path.join(results_path, "training", "model", "autoencoder" + "pruned_fine_tuned.pt")
            torch.save(fine_tuned_state, pruned_finetune_model_path)
            #load_sparse_model(pruned_fine_tuned_model_path, model)
            pruned_fine_tuned_model=load_and_evaluate_pruned_model(config, pruned_finetune_model_path )

            print('****************Result of fine-tuning of pruned model ******************')

            gc.collect()
            time.sleep(5)  
            tracemalloc.start() 
            snapshot_before = tracemalloc.take_snapshot()

            t0 = time.perf_counter()
            initial_cpu_usage = get_cpu_usage()
            power_usage = estimate_power_usage(initial_cpu_usage)

            pruned_finetune_model_accuracy, test_ap=pruned_fine_tuned_model.autoencoder.single_test(train_data, test_data)

            pruned_finetune_cpu_usage = get_cpu_usage()
            t1 = time.perf_counter()
            t_pruned_finetune_model=t1-t0

            snapshot_after = tracemalloc.take_snapshot()
            tracemalloc.stop()
            top_stats = snapshot_after.compare_to(snapshot_before, 'lineno')

            pruned_finetune_total_memory_diff = sum([stat.size_diff for stat in top_stats])
            pruned_finetune_energy_consumption = power_usage * t_pruned_finetune_model
            pruned_finetune_model_size = os.path.getsize( pruned_finetune_model_path)
            num_parm_pruned_finetune_model=get_num_parameters(pruned_fine_tuned_model, count_nonzero_only=True)

            gc.collect()
            time.sleep(5)  # Add a 5-second delay to stabilize the initial state    

            ###### Report  

            print(f"pruned_finetune model has accuracy on test set={pruned_finetune_model_accuracy:.2f}%")
            print(f"pruned_finetune model has size={pruned_finetune_model_size:.2f} ")
            print(f"The time inference of pruned_finetune model is ={t_pruned_finetune_model}") 
            print(f"The number of parametrs of pruned_finetune model is:{num_parm_pruned_finetune_model}") 

            print(f"Energy Consumption of pruned_finetune model: {pruned_finetune_energy_consumption:.3f}")
            print(f"total memory usage of pruned_finetune model':{pruned_finetune_total_memory_diff} ")
            print(f"cpu usage of pruned_finetune model':{pruned_finetune_cpu_usage:.3f} %")


            #Update my Eva dictionary
            Eva.update({'pruned and finetune model accuracy': pruned_finetune_model_accuracy,
                        'time inference of pruned and finetune model': t_pruned_finetune_model,
                        'number parmameters of pruned and finetune model': num_parm_pruned_finetune_model,
                        'size of pruned and finetune model': pruned_finetune_model_size, 
                        'energy consumption of pruned and finetune model':pruned_finetune_energy_consumption,
                        'total memory usage of pruned and finetune model':pruned_finetune_total_memory_diff,
                        'cpu usage of pruned and finetune model':pruned_finetune_cpu_usage
                       })

            gc.collect()
            time.sleep(5) 

            Base_model_accuracy.append(Eva['base model accuracy'])
            T_base_model.append(Eva['time inference of base model'])
            Num_parm_base_model.append(int(Eva['number parmameters of base model']))
            Base_model_size.append(int(Eva['size of base model']))
            Base_Energy_Consumption.append(Eva['energy consumption of base model'])
            Base_Cpu_Usage.append(Eva['cpu usage of base model'])
            Base_Memory_Usage.append(Eva['total memory usage of base model'])

            Pruned_model_accuracy.append(Eva['pruned model accuracy'])
            T_pruned_model.append(Eva['time inference of pruned model'])
            Num_parm_pruned_model.append(int(Eva['number parmameters of pruned model']))
            Pruned_model_size.append(int(Eva['size of pruned model']))
            Pruned_Energy_Consumption.append(Eva['energy consumption of pruned model'])
            Pruned_Cpu_Usage.append(Eva['cpu usage of pruned model'])
            Pruned_Memory_Usage.append(Eva['total memory usage of pruned model'])


            Pruned_finetune_model_accuracy.append(Eva['pruned and finetune model accuracy'])
            T_pruned_finetune_model.append(Eva['time inference of pruned and finetune model'])
            Num_parm_pruned_finetune_model.append(int(Eva['number parmameters of pruned and finetune model']))
            Pruned_finetune_model_size.append(int(Eva['size of pruned and finetune model']))
            Pruned_finetune_Energy_Consumption.append(Eva['energy consumption of pruned and finetune model'])
            Pruned_finetune_Cpu_Usage.append(Eva['cpu usage of pruned and finetune model'])
            Pruned_finetune_Memory_Usage.append(Eva['total memory usage of pruned and finetune model'])


    Eva_final=dict()
    base_model_accuracy_mean = stat.mean(Base_model_accuracy)
    base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
    #desc = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean,base_model_accuracy_std)


    Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
    Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})

    t_base_model_mean =stat.mean(T_base_model)
    t_base_model_std =stat.stdev(T_base_model)  
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
    Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})


    num_parm_base_model_mean = stat.mean(Num_parm_base_model)
    num_parm_base_model_std = stat.stdev(Num_parm_base_model)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
    Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})

    base_model_size_mean = stat.mean(Base_model_size)
    base_model_size_std = stat.stdev(Base_model_size)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of base model size':base_model_size_mean})
    Eva_final.update({'Std of base model size':base_model_size_std})


    base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
    base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
    Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})


    base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
    base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
    Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})

    base_memory_usage_mean = stat.mean(Base_Memory_Usage)
    base_memory_usage_std = stat.stdev(Base_Memory_Usage)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
    Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})




    #################################

    pruned_model_accuracy_mean =stat.mean(Pruned_model_accuracy)
    pruned_model_accuracy_std = stat.stdev(Pruned_model_accuracy)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of pruned model accuracy':float(format(pruned_model_accuracy_mean, '.3f'))})
    Eva_final.update({'Std of pruned model accuracy':float(format(pruned_model_accuracy_std, '.3f'))})


    t_pruned_model_mean = stat.mean(T_pruned_model)
    t_pruned_model_std =stat.stdev(T_pruned_model)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of time inference of pruned model':float(format(t_pruned_model_mean, '.3f'))})
    Eva_final.update({'Std of time inference of pruned model':float(format(t_pruned_model_std, '.3f'))})

    num_parm_pruned_model_mean = stat.mean(Num_parm_pruned_model)
    num_parm_pruned_model_std = stat.stdev(Num_parm_pruned_model)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of number parmameters of pruned model':num_parm_pruned_model_mean})
    Eva_final.update({'Std of number parmameters of pruned model':num_parm_pruned_model_std})

    pruned_model_size_mean =stat.mean( Pruned_model_size)
    pruned_model_size_std = stat.stdev(Pruned_model_size)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of pruned model size':pruned_model_size_mean})
    Eva_final.update({'Std of pruned_model_size':pruned_model_size_std })

    pruned_energy_consumption_mean = stat.mean(Pruned_Energy_Consumption)
    pruned_energy_consumption_std = stat.stdev(Pruned_Energy_Consumption)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of energy consumption of pruned model':pruned_energy_consumption_mean })
    Eva_final.update({'Std of energy consumption of pruned model':pruned_energy_consumption_std})


    pruned_cpu_usage_mean = stat.mean(Pruned_Cpu_Usage)
    pruned_cpu_usage_std = stat.stdev(Pruned_Cpu_Usage)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of cpu usage of pruned model':pruned_cpu_usage_mean})
    Eva_final.update({'Std of cpu usage of pruned model':pruned_cpu_usage_std})

    pruned_memory_usage_mean = stat.mean(Pruned_Memory_Usage)
    pruned_memory_usage_std = stat.stdev(Pruned_Memory_Usage)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of memory usage of pruned model':pruned_memory_usage_mean})
    Eva_final.update({'Std of memory usage of pruned model':pruned_memory_usage_std})


    #################################
    pruned_finetune_model_accuracy_mean =stat.mean(Pruned_finetune_model_accuracy)
    pruned_finetune_model_accuracy_std = stat.stdev(Pruned_finetune_model_accuracy)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_mean, '.3f'))})
    Eva_final.update({'Std of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_std, '.3f'))})                 

    t_pruned_finetune_model_mean =stat.mean(T_pruned_finetune_model)
    t_pruned_finetune_model_std =stat.stdev(T_pruned_finetune_model)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of time inference of pruned finetune model':float(format(t_pruned_finetune_model_mean,'.3f'))})
    Eva_final.update({'Std of time inference of pruned finetune model':float(format(t_pruned_finetune_model_std,'.3f'))})

    num_parm_pruned_finetune_model_mean =stat.mean(Num_parm_pruned_finetune_model)
    num_parm_pruned_finetune_model_std = stat.stdev(Num_parm_pruned_finetune_model)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_mean})
    Eva_final.update({'Std of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_std })

    pruned_finetune_model_size_mean = stat.mean(Pruned_finetune_model_size)
    pruned_finetune_model_size_std = stat.stdev(Pruned_finetune_model_size)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of pruned finetune model size':pruned_finetune_model_size_mean})
    Eva_final.update({'Std of pruned finetune model size':pruned_finetune_model_size_std})


    pruned_finetune_energy_consumption_mean = stat.mean(Pruned_finetune_Energy_Consumption)
    pruned_finetune_energy_consumption_std = stat.stdev(Pruned_finetune_Energy_Consumption)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_mean })
    Eva_final.update({'Std of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_std})


    pruned_finetune_cpu_usage_mean = stat.mean(Pruned_finetune_Cpu_Usage)
    pruned_finetune_cpu_usage_std = stat.stdev(Pruned_finetune_Cpu_Usage)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_mean})
    Eva_final.update({'Std of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_std})

    pruned_finetune_memory_usage_mean = stat.mean(Pruned_finetune_Memory_Usage)
    pruned_finetune_memory_usage_std = stat.stdev(Pruned_finetune_Memory_Usage)
    #desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
    Eva_final.update({'Ave of memory usage of pruned_finetune model':pruned_finetune_memory_usage_mean})
    Eva_final.update({'Std of memory usage of pruned_finetune model':pruned_finetune_memory_usage_std})






    ### The sparsity changes across range(0, 1, .01)
    dataset_name = 'Cora_Link'
    Pruning_Method='Grain_Pruning'
    max_epoch = 100
    resume = True
    result_folder ='pathresult/'
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)



    file_name = result_folder+Pruning_Method+'_'+'with sparsity of'+'_'+str(sparsity)+'_on_'+dataset_name+'_'+str(max_epoch)+'.txt'

    with open(file_name, 'w') as f:
            f.write('%s:%s\n'%('dataset_name', 'Cora_Link'))
            f.write('%s:%s\n'%('max_epoch', max_epoch))
            f.write('%s:%s\n'%('sparsity', sparsity))
            for key, value in Eva_final.items():
                f.write('%s:%s\n'%(key, value))

            for key, value in Eva_measure.items():
                f.write('%s:%s\n' % (key, ','.join(map(str, value))))                    


________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

________________________________________
This is iteration 0
Training and evaluation before pruning 
Directories are set.
Directories are set.
Building the models for training and evaluation in NESS framework...
----------------------------------------Summary of the models:----------------------------------------
================================== NESS Architecture ==================================
GAEWrapper(
  (gae): GAE(
    (encoder): GNAEEncoder(
      (linear1): Linear(in_features=1433, out_features=32, bias=True)
      (propagate): APPNP(K=1, alpha=0)
    )
    (decoder): InnerProductDecoder()
  )
)

is_pruned:False
Epoch:[-1], Total loss:7.8579, X recon loss:3.1438, val auc:NA
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[39] training loss:2.3546, X recon loss:0.9534, val auc:0.8978877820989171
Done with saving models.
Done with saving models.
Done with saving models.
Done with saving models.
Epoch:[79] training loss:2.2055, X recon loss:0.8

<Figure size 432x288 with 0 Axes>

### Computing the mean and std 

In [ ]:
Eva_final=dict()
base_model_accuracy_mean = stat.mean(Base_model_accuracy)
base_model_accuracy_std =  stat.stdev(Base_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(base_model_accuracy_mean,base_model_accuracy_std)


Eva_final.update({'Ave of base model accuracy':float(format(base_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of base model accuracy':float(format(base_model_accuracy_std, '.3f'))})
                 
t_base_model_mean =stat.mean(T_base_model)
t_base_model_std =stat.stdev(T_base_model)  
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of time inference of base model':float(format(t_base_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of base model':float(format(t_base_model_std, '.3f'))})


num_parm_base_model_mean = stat.mean(Num_parm_base_model)
num_parm_base_model_std = stat.stdev(Num_parm_base_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of number parmameters of base model':num_parm_base_model_mean})
Eva_final.update({'Std of number parmameters of base model':num_parm_base_model_std})

base_model_size_mean = stat.mean(Base_model_size)
base_model_size_std = stat.stdev(Base_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of base model size':base_model_size_mean})
Eva_final.update({'Std of base model size':base_model_size_std})


base_energy_consumption_mean = stat.mean(Base_Energy_Consumption)
base_energy_consumption_std = stat.stdev(Base_Energy_Consumption)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of energy consumption of base model':base_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of base model':base_energy_consumption_std})


base_cpu_usage_mean = stat.mean(Base_Cpu_Usage)
base_cpu_usage_std = stat.stdev(Base_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of base model':base_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of base model':base_cpu_usage_std})

base_memory_usage_mean = stat.mean(Base_Memory_Usage)
base_memory_usage_std = stat.stdev(Base_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of base model':base_memory_usage_mean})
Eva_final.update({'Std of memory usage of base model':base_memory_usage_std})




#################################

pruned_model_accuracy_mean =stat.mean(Pruned_model_accuracy)
pruned_model_accuracy_std = stat.stdev(Pruned_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned model accuracy':float(format(pruned_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of pruned model accuracy':float(format(pruned_model_accuracy_std, '.3f'))})
                 

t_pruned_model_mean = stat.mean(T_pruned_model)
t_pruned_model_std =stat.stdev(T_pruned_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of time inference of pruned model':float(format(t_pruned_model_mean, '.3f'))})
Eva_final.update({'Std of time inference of pruned model':float(format(t_pruned_model_std, '.3f'))})

num_parm_pruned_model_mean = stat.mean(Num_parm_pruned_model)
num_parm_pruned_model_std = stat.stdev(Num_parm_pruned_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of number parmameters of pruned model':num_parm_pruned_model_mean})
Eva_final.update({'Std of number parmameters of pruned model':num_parm_pruned_model_std})

pruned_model_size_mean =stat.mean( Pruned_model_size)
pruned_model_size_std = stat.stdev(Pruned_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned model size':pruned_model_size_mean})
Eva_final.update({'Std of pruned_model_size':pruned_model_size_std })

pruned_energy_consumption_mean = stat.mean(Pruned_Energy_Consumption)
pruned_energy_consumption_std = stat.stdev(Pruned_Energy_Consumption)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of energy consumption of pruned model':pruned_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of pruned model':pruned_energy_consumption_std})


pruned_cpu_usage_mean = stat.mean(Pruned_Cpu_Usage)
pruned_cpu_usage_std = stat.stdev(Pruned_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of pruned model':pruned_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of pruned model':pruned_cpu_usage_std})

pruned_memory_usage_mean = stat.mean(Pruned_Memory_Usage)
pruned_memory_usage_std = stat.stdev(Pruned_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of pruned model':pruned_memory_usage_mean})
Eva_final.update({'Std of memory usage of pruned model':pruned_memory_usage_std})


#################################
pruned_finetune_model_accuracy_mean =stat.mean(Pruned_finetune_model_accuracy)
pruned_finetune_model_accuracy_std = stat.stdev(Pruned_finetune_model_accuracy)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_mean, '.3f'))})
Eva_final.update({'Std of pruned finetune model accuracy':float(format(pruned_finetune_model_accuracy_std, '.3f'))})                 

t_pruned_finetune_model_mean =stat.mean(T_pruned_finetune_model)
t_pruned_finetune_model_std =stat.stdev(T_pruned_finetune_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of time inference of pruned finetune model':float(format(t_pruned_finetune_model_mean,'.3f'))})
Eva_final.update({'Std of time inference of pruned finetune model':float(format(t_pruned_finetune_model_std,'.3f'))})

num_parm_pruned_finetune_model_mean =stat.mean(Num_parm_pruned_finetune_model)
num_parm_pruned_finetune_model_std = stat.stdev(Num_parm_pruned_finetune_model)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_mean})
Eva_final.update({'Std of number parmameters of pruned finetune model':num_parm_pruned_finetune_model_std })

pruned_finetune_model_size_mean = stat.mean(Pruned_finetune_model_size)
pruned_finetune_model_size_std = stat.stdev(Pruned_finetune_model_size)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of pruned finetune model size':pruned_finetune_model_size_mean})
Eva_final.update({'Std of pruned finetune model size':pruned_finetune_model_size_std})


pruned_finetune_energy_consumption_mean = stat.mean(Pruned_finetune_Energy_Consumption)
pruned_finetune_energy_consumption_std = stat.stdev(Pruned_finetune_Energy_Consumption)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_mean })
Eva_final.update({'Std of energy consumption of pruned_finetune model':pruned_finetune_energy_consumption_std})


pruned_finetune_cpu_usage_mean = stat.mean(Pruned_finetune_Cpu_Usage)
pruned_finetune_cpu_usage_std = stat.stdev(Pruned_finetune_Cpu_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_mean})
Eva_final.update({'Std of cpu usage of pruned_finetune model':pruned_finetune_cpu_usage_std})

pruned_finetune_memory_usage_mean = stat.mean(Pruned_finetune_Memory_Usage)
pruned_finetune_memory_usage_std = stat.stdev(Pruned_finetune_Memory_Usage)
#desc = "{:.3f} ± {:.3f}".format(acc_mean,acc_std)
Eva_final.update({'Ave of memory usage of pruned_finetune model':pruned_finetune_memory_usage_mean})
Eva_final.update({'Std of memory usage of pruned_finetune model':pruned_finetune_memory_usage_std})





#################################


print(f"All measurement about pruning process of sparsity:{sparsity*100}% ")   
Eva_final

### Recording results on txt file

In [23]:

### The sparsity changes across range(0, 1, .01)
dataset_name = 'Cora_Link'
Pruning_Method='Grain_Pruning'
max_epoch = 100
resume = True
result_folder ='pathresult/'
if not os.path.exists(result_folder):
    os.makedirs(result_folder)



file_name = result_folder+Pruning_Method+'_'+'with sparsity of'+'_'+str(sparsity)+'_on_'+dataset_name+'_'+str(max_epoch)+'.txt'

with open(file_name, 'w') as f:
        f.write('%s:%s\n'%('dataset_name', 'Cora_Link'))
        f.write('%s:%s\n'%('max_epoch', max_epoch))
        f.write('%s:%s\n'%('sparsity', sparsity))
        for key, value in Eva_final.items():
            f.write('%s:%s\n'%(key, value))
            
        for key, value in Eva_measure.items():
            f.write('%s:%s\n' % (key, ','.join(map(str, value))))                    
       